Скачаем обученную модель с Google drive

In [ ]:
!gdown --id 1yDUEkeqfDGaSOgKFP_1OVayCrPm91fBk -O ivision_model.zip

In [20]:
!mkdir ivision_model

In [18]:
import cv2
import os
import numpy as np
import tensorflow as tf
import zipfile

In [21]:
with zipfile.ZipFile('ivision_model.zip', 'r') as zip_ref:
    zip_ref.extractall('ivision_model')

In [88]:
RESIZE_SHAPE = (224, 224) #размерность кадров после сжатия
FRAME_LABEL_THR = 0.025
CAR_ACCIDENT_THR = 0.4
test_videos_path = 'videos/' # измените данную переменную на путь до папки с тестовым видео

In [40]:
def preprocess_videos(data_path: str):
    """
    Extracts every fifth frame from every video in data_dir 
    and reshapes it in (224, 224) format.
    
    Parameters
    ==========
    data_path: string
    Path to folder with videos
    
    Returns
    ======= 
    preprocessed_data: Dict[str -> numpy.array]
    Mapping between video name and sampled video frames
    """
    preprocessed_data = {}
    for filename in list(os.walk(data_path))[0][2]:
        vidcap = cv2.VideoCapture(data_path+filename)
        count = 0
        preprocessed_data[filename] = []
        while True:
            ret, frame = vidcap.read()
            if not ret:
                break
            if not count % 5:
                resized_frame = cv2.resize(frame, RESIZE_SHAPE, interpolation = cv2.INTER_AREA)
                preprocessed_data[filename].append(resized_frame)
            count += 1
        preprocessed_data[filename] = np.array(preprocessed_data[filename])
        vidcap.release()
        print(f"{filename} has been processed")
    return preprocessed_data

In [78]:
test_data = preprocess_videos(data_path=test_videos_path)

12.mp4 has been processed
19.mp4 has been processed
26.mp4 has been processed
33.mp4 has been processed
48.mp4 has been processed


In [22]:
model = tf.keras.models.load_model('ivision_model/ivision_model')

In [89]:
results = {}

In [96]:
dtp_flag = ['No car accident', 'Car accident']
for video_name, frames in test_data.items():
    frame_probas = tf.keras.activations.sigmoid(model.predict(frames)).numpy()
    frame_labels = (frame_probas > FRAME_LABEL_THR)*1
    total_accident_frames = sum(frame_labels)
    results[video_name] = ((total_accident_frames > CAR_ACCIDENT_THR*len(frame_labels))*1)[0]
    print(f"{video_name}: {dtp_flag[results[video_name]]}")

12.mp4: Car accident
19.mp4: No car accident
26.mp4: No car accident
33.mp4: Car accident
48.mp4: No car accident
